### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.optimize as op
import scipy.io

### Importing Data

In [7]:
# mat = scipy.io.loadmat('matrix.mat')
# matrix = mat['srw_mat']
matrix = np.load('./srw.npy', allow_pickle=True)

In [9]:
matrix = matrix.reshape(11473, 742)

### Creating graph representation and Feauture Matrix (psi)

In [10]:
m,n = matrix.shape
Adj = np.zeros((1,n))
psi = np.zeros((1,n,1))
for j in range(n):
    if matrix[0][j] >=1:
        Adj[0][j] = 1
        psi[0][j][0] = np.squeeze(matrix[0][j])

### Functions for SRW

In [11]:
def sigmoid(z,b):
  """
  this function returns the wilcoxon loss function of each element in the input matrix
   diff = input matrix
   b = wilcoxon parameter
   loss = output matrix
  """
  return (1 / (1 + np.exp(-z/b)))

In [13]:
def FeaturesToEdgeStrength(psi, w):
    
    m = psi.shape[0]
    n = psi.shape[1]
    
    p = w.shape[0]
    
    
    A = np.zeros(shape=(m, n))
    dA = np.zeros(shape=(m, n, p))
    
    wmatrix = np.ones((m,n,p))
    for i in range(p):
        wmatrix[:, :, i] *= w[i]
        
    A = sigmoid(np.sum(np.multiply(psi,wmatrix),axis=-1),1)
    temp = np.multiply(A,1-A)
    
    for i in range(p):
        dA[:,:,i] = temp

    dA = np.multiply(dA,wmatrix)
    
    return A,dA

In [14]:
def EdgeStrengthToTransitionProbability(A,alpha):
    
    m,n = A.shape
    rowsum = np.sum(A,axis=1)
    Q = np.divide(A,rowsum)
    
    return Q

In [16]:
def EdgeStrengthToPartialdiffTransition(A,dA,alpha):
    m = A.shape[0]
    if len(dA.shape)==3: 
        p = dA.shape[2];
    else:
        p = 1;

    dQ = np.zeros((m,n,p))
    sumU = np.sum(A,axis=1); # sum of transition prob of all nodes starting from U
    den = sumU**2
    for i in range(p):
        sumdU = np.sum(dA[:,:,i],axis=1) # sum of partial d of trans p of all nodes
        temp = np.multiply(sumU,dA[:,:,i]) + np.multiply(A,sumdU)
        dQ[:,:,i] = np.divide(temp,den)

    return dQ

In [17]:
def Converged(p1,p2,eps):
    
    temp = np.max(np.abs(p1-p2))
    
    if temp < eps:
        ret = 1
    else:
        ret = 0
    
    return ret

In [19]:
def ComputeStationaryP(Q, dQ):

    epsilon = 1e-12
    m,n = Q.shape
    
    if len(dQ.shape)==3:
        p = (dQ.shape)[2]
    else:
        p = 1
        
    P = np.zeros((m,n))
    dP = np.zeros((n,p))
    P = P + (1/n)
    t = 0

    while True:
        t += 1
        Pnew = P
        for i in range(n):
            Pnew[0][i] = np.sum(np.multiply(P[0][i],Q[0,i]))
            
        Pnew = np.divide(Pnew,np.sum(Pnew,axis=1))
        temp = P
        P = Pnew
        if ((Converged(P, temp, epsilon) or t>100)):
            break
            
    for j in range(p):
        k = 0;
        dQj = dQ[:,:,j]
        while True:
            k += 1
            dpjnew = dP[:,j]
            for i in range(n):
                dpjnew[i] = (Q[0][i]*dP[i,j]) + P[0][i]*dQj[:,i]  
            temp =dP[:,j]
            dP[:,j] = dpjnew
            if (Converged(dP[:,j], temp, epsilon) or k>100):
                break

    return P,dP

In [26]:
def DifferenceIndices(P,d):
    n = P.shape[1]
    a = np.vstack([P]*n)
    b = np.hstack([P.T]*n) 

    dd = np.subtract(a,b)
  

    dd[d!=0,:] = 0; dd[:,d==0] = 0;

    index = (dd>0)
    count = 0
    I = []
    for i in range(742):
        for j in range(742):
            if index[i][j] ==True:
                I.append((i,j))
    I = np.array(I)

    return I,dd

In [27]:
def dsigmoid(diff, wmvloss, b):
    
    temp = 1 - wmvloss
    der  = np.multiply(wmvloss,temp)
    der /= b
  
    return der

In [28]:
def LossFunction(param,features,d,lmbd=1,alpha=0.2,b=0.4):
    
    p = len(param)
    n = len(d)
    J = 0
    
    grad = np.zeros((1,p))

    A, dA = FeaturesToEdgeStrength(features, param)
    
    Q = EdgeStrengthToTransitionProbability(A, alpha)
    dQ = EdgeStrengthToPartialdiffTransition(A, dA, alpha)
    P, dP = ComputeStationaryP(Q, dQ)

    I, diff = DifferenceIndices(P, d)
    
    diff[diff<0] = 0
  
    wmvloss = sigmoid(diff,b)
    J = np.sum(param**2) + lmbd* np.sum(wmvloss)

    return J

In [29]:
def gradient(param,features,d,lmbd=1,alpha=0.2,b=0.4):

    p = len(param)
    n = len(d)
    J = 0
    grad = np.zeros((1,p))
    
    A, dA = FeaturesToEdgeStrength(features, param)
    Q = EdgeStrengthToTransitionProbability(A, alpha)
    dQ = EdgeStrengthToPartialdiffTransition(A, dA, alpha)
    P, dP = ComputeStationaryP(Q, dQ)
    I, diff = DifferenceIndices(P, d)
    diff[diff<0] = 0
  
  
    wmvloss = sigmoid(diff,b)

    J = np.sum(param**2) + lmbd* np.sum(wmvloss)
    derivative = dsigmoid(diff,wmvloss,b)

    for i in range(p):
        grad[i] += 2*param[i]
        
        temp = 0

        for j in range(I.shape[0]):
            ind = I[j,:]
            temp += derivative[ind[0],ind[1]] * (dP[ind[0],i]-dP[ind[1],i])
            
        grad[i] += temp*lmbd

    grad = grad.T
    
    return grad

### Running the optimizer for Analysis

In [31]:
m = psi.shape[0]
n = psi.shape[1]
p = psi.shape[2]

initial_w = np.ones((1,p))
assert(initial_w.shape == (1,p))
d = Adj[0,:]

loss = LossFunction(initial_w, psi, d, lmbd=1,alpha=0.2,b=0.4)

result = sp.optimize.fmin_cg(
    LossFunction,
    initial_w,
    fprime = gradient,
    args = (psi, d,0.2,0.2,0.4),
    full_output = True,
    retall = True,
    epsilon = 1.5e-15,
    maxiter = 50
)

Optimization terminated successfully.
         Current function value: 55056.400000
         Iterations: 1
         Function evaluations: 8
         Gradient evaluations: 8
